## B. Data Exploration

In this phase, we will answer the following questions:
1. How many rows and how many columns?
2. What is the meaning of each row?
3. Are there duplicated rows?
4. What is the meaning of each column?
5. What is the current data type of each column? Are there columns having
inappropriate data types?
6. With each numerical column, how are values distributed?
    - What is the percentage ofmissing values?
    - Min? max? Are they abnormal?
7. With each categorical column, how are values distributed?
    - What is the percentage of missing values?
    - How many different values? Show a few Are they abnormal?

---
### Import libraries nesscessary 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Read data from csv file into Pandas dataframe
rented_house_df = pd.read_csv('../data/HCMHouseRentPreprocessing.csv', sep=',')
rented_house_df.head()

,title,price,published,acreage,street,ward,district
0,"Cho thuê nhà trọ mới sạch đẹp tại Lê Đình Cẩn,...",2200000,2022-05-16,20.0,Lê Đình Cẩn,Phường Tân Tạo,Quận Bình Tân
1,Cho thuê phòng trọ giá rẻ ở mặt tiền hẻm lớn Đ...,2500000,2022-04-20,20.0,487/35/25 Đường Huỳnh Tấn Phát,Phường Tân Thuận Đông,Quận 7
2,Cho thuê phòng trọ kdc Nam Long-Trần Trọng Cun...,3500000,2022-05-10,30.0,Đường 10,Phường Tân Thuận Đông,Quận 7
3,Phòng trọ giá rẻ ngay cổng khu chế xuất Tân Th...,1500000,2022-05-05,30.0,283/15 Huỳnh Tấn Phát,Phường Tân Thuận Đông,Quận 7
4,"Cho thuê phòng có gác, không gác, tolet riêng ...",3500000,2022-01-05,18.0,Lê Văn Sỹ,Phường 14,Quận Phú Nhuận


### 1. How many rows and how many columns?

In [3]:
num_rows = rented_house_df.shape[0]
num_cols = rented_house_df.shape[1]
print('Number of rows: ', num_rows)
print('Number of columns: ', num_cols)

Number of rows:  8948
Number of columns:  7


### 2. What is the meaning of each row?

A line indicates the information about renting a house in Ho Chi Minh City. Each line provides prices, Acreage in square meter, published date and the address of the house.


### 3. Are there duplicated rows?

In [4]:
num_duplicated_rows = rented_house_df.index.duplicated(keep='first').sum()
num_duplicated_rows

0

### 4. What is the meaning of each column?

Here is the description about the columns in the file "HCMHouseRentPreprocessing.csv":
- **title**: the title of the rented houses
- **price**: price of the rented houses (if price is -1, it mean the owner want to disscuss more)
- **acreage**:  acreage in square meter of the rented houses
- **street**: the street of the rented houses
- **ward**: the ward of the rented houses
- **district**: the district of the rented houses
    

### 5. What is the current data type of each column? Are there columns having inappropriate data types?

In [5]:
col_dtypes = rented_house_df.dtypes
col_dtypes

title         object
price          int64
published     object
acreage      float64
street        object
ward          object
district      object
dtype: object

What does columns that have object dtype mean?**
There are 4 columns with the datatype of `object`: "published", "street", "ward", "district".

In [6]:
def open_object_dtype(s):
    dtypes = set()

    dtypes = set(s.apply(type))
    
    return dtypes

In [7]:
open_object_dtype(rented_house_df['published'])

{str}

Issues that need to be reprocessed:

- The column "published" has `str` data type. To further explore this column, we will perform the preprocessing step of converting it to `datetime` data type.

In [8]:
# Convert dtype of "published" column to datetime
rented_house_df['published'] = pd.to_datetime(rented_house_df['published'], format='%Y/%m/%d')
rented_house_df.head()


,title,price,published,acreage,street,ward,district
0,"Cho thuê nhà trọ mới sạch đẹp tại Lê Đình Cẩn,...",2200000,2022-05-16,20.0,Lê Đình Cẩn,Phường Tân Tạo,Quận Bình Tân
1,Cho thuê phòng trọ giá rẻ ở mặt tiền hẻm lớn Đ...,2500000,2022-04-20,20.0,487/35/25 Đường Huỳnh Tấn Phát,Phường Tân Thuận Đông,Quận 7
2,Cho thuê phòng trọ kdc Nam Long-Trần Trọng Cun...,3500000,2022-05-10,30.0,Đường 10,Phường Tân Thuận Đông,Quận 7
3,Phòng trọ giá rẻ ngay cổng khu chế xuất Tân Th...,1500000,2022-05-05,30.0,283/15 Huỳnh Tấn Phát,Phường Tân Thuận Đông,Quận 7
4,"Cho thuê phòng có gác, không gác, tolet riêng ...",3500000,2022-01-05,18.0,Lê Văn Sỹ,Phường 14,Quận Phú Nhuận


### 6. With each numerical column, how are values distributed?
        - What is the percentage of missing values?
        - Min? max? Are they abnormal?

- What is the percentage of missing values?

In [9]:
num_cols_df = pd.DataFrame(columns=rented_house_df.columns.drop(['title', 'published', 'street', 'ward', 'district']))

num_missing_val = rented_house_df[num_cols_df.columns].isnull().sum()

num_cols_df.loc['missing_ratio'] = num_missing_val / num_rows * 100

num_cols_df


,price,acreage
missing_ratio,0.0,0.0


- Min? max? Are they abnormal?

In [10]:
rented_house_df.describe()

,price,acreage
count,8.948000e+03,8948.000000
mean,3.423750e+06,26.326056
std,3.388884e+06,29.888507
min,-1.000000e+06,0.000000
25%,2.300000e+06,20.000000
50%,3.200000e+06,25.000000
75%,4.000000e+06,30.000000
max,1.500000e+08,1000.000000


- Because the acreage of the rented houses can not equal 0. So we need to remove them.

In [11]:
# remove the rows with acreage is 0
rented_house_df = rented_house_df[rented_house_df['acreage'] != 0]

In [12]:
rented_house_df.describe()

,price,acreage
count,8.941000e+03,8941.000000
mean,3.425055e+06,26.346667
std,3.389699e+06,29.891123
min,-1.000000e+06,2.000000
25%,2.300000e+06,20.000000
50%,3.200000e+06,25.000000
75%,4.000000e+06,30.000000
max,1.500000e+08,1000.000000


### 7. With each categorical column, how are values distributed?
        - What is the percentage of missing values?
        - How many different values? Show a few Are they abnormal?

In [13]:
cate_cols_df = pd.DataFrame(columns=rented_house_df.columns.drop(['price', 'acreage']))

num_missing_val = rented_house_df[cate_cols_df.columns].isnull().sum()

cate_cols_df.loc['missing_ratio'] = num_missing_val / num_rows * 100

cate_cols_df.loc['num_diff_vals'] = rented_house_df.apply(lambda x: x.nunique())

cate_cols_df.loc['diff_vals'] = rented_house_df.apply(lambda x: x[~x.isnull()].unique())

cate_cols_df

,title,published,street,ward,district
missing_ratio,0.0,0.0,0.0,0.0,0.0
num_diff_vals,8935.0,1671.0,5628.0,166.0,24.0
diff_vals,[Cho thuê nhà trọ mới sạch đẹp tại Lê Đình Cẩn...,"[2022-05-16T00:00:00.000000000, 2022-04-20T00:...","[Lê Đình Cẩn, 487/35/25 Đường Huỳnh Tấn Phát, ...","[Phường Tân Tạo, Phường Tân Thuận Đông, Phường...","[Quận Bình Tân, Quận 7, Quận Phú Nhuận, Quận 3..."
